In [ ]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import urllib
import pandas as pd
import re
import datetime
import numpy as np
import pickle
import requests
from time import sleep

In [ ]:
version = "1.0.0"

# Get The Article list

In [ ]:


from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import ui
from selenium.webdriver.support.ui import WebDriverWait, Select

from selenium.webdriver.firefox.options import Options
options = Options()
options.headless = True
driver = webdriver.Firefox(options=options)
driver.set_window_size(1920, 1080)



In [ ]:
def gedjournalarticles(journal, filename ):
    start_url = "https://academic.oup.com/journals/advanced-search"
    driver.get(start_url)
    print(driver.current_url)
    sleep(4)
    
    content = driver.page_source
    
    # open old file
    olddf = pd.DataFrame({'url': [], 'journal_title': [],
                                'title': [], 'writer': [], 
                                'doi': [], 'abstract': [],
                                'keyword': [], 'publishdate': [],
                                'language': [],
                                'category': [],
                                'journal_eissn': [], 'journal_pissn': []})
    olddf.set_index('doi', inplace=True)
    olddf['doi'] = []
    try:
        olddf = pd.read_pickle(filename)
        print(olddf.tail())
    except:
        pass
    
    df =  pd.DataFrame({'url': [], 'journal_title': [],
                                'title': [], 'writer': [], 
                                'doi': [], 'abstract': [],
                                'keyword': [], 'publishdate': [],
                                'language': [],
                                'category': [],
                                'journal_eissn': [], 'journal_pissn': []})
    df.set_index('doi', inplace=True)
    df['doi'] = []

    # set journal
    try:
        selectpublisher = Select(driver.find_element_by_id("SelectedJournal"))
    except Exception as e:
        print("Error in SelectedJournal: ", e)
        return df, olddf
    print(journal['title'].split(":")[0])
    searchtitle = journal['title'].split(":")[0]
    try:        
        # patch for specific articles
        if searchtitle == 'AoB Plants':
            searchtitle = 'AoB PLANTS'
        
        selectpublisher.select_by_visible_text(searchtitle)                        
    except Exception as e:
        print("Error in journal selection:", e)
        return df, olddf
    a = driver.find_element_by_id("btnAdvancedCitationFilter")
    if a is not None:
        a.click()
    sleep(4)
    print(driver.current_url)

    # just for Research articles
    try:
        driver.find_element_by_id("article-type-label0").click()
    except Exception as e:
        print(e)        
    
    def processresult(df):
        sleep(np.random.uniform(3,60))        
        
    
        # open all of the abstract
        for selabstract in driver.find_elements_by_class_name('js-show-abstract'):
            selabstract.click()
            sleep(np.random.uniform(3,120))
        
        # BeautifulSoup
        content = driver.page_source
        soup = BeautifulSoup(content, features="lxml")
        # get the articles 
        articles = soup.find_all('div', {'class': ['al-article-box']})
        

    
        for article in articles:
            title = article.find("h4").get_text()
            url = article.find("h4").find("a")['href']
            print("url", url)
            date = article.find('div', {'class': ['al-pub-date']}).get_text().replace("Published: ", '')
            date = datetime.datetime.strptime(date, "%d %B %Y").strftime('%Y-%m-%d')        
            abstract = article.find('section', {'class': ['abstract']}).get_text()
            abstract = re.sub(searchtitle+' URL: .+', '', abstract )
            doi = ""
            doidiv = article.find('div', {'class': ['al-citation-list']}).find_all('a')
            for a in doidiv:
                if re.search("doi.org", a["href"] ):
                    doi = a["href"].replace("https://doi.org/", '')
                    break
            writers = []
            writersdeiv =  article.find('div', {'class': ['al-authors-list']}).find_all( 'span', {'class': ['wi-fullname']} )
            for w in writersdeiv:
                writers.append(w.get_text())
            writers = "#".join(writers)
        
            # add data to the df
            datadict = pd.Series({
                        'journal_title': journal['title'],
                        'url': url,
                        'title': title,
                        'writer': writers,
                        'doi': doi,
                        'abstract': abstract,
                        'keyword': "",
                        'publishdate': date,
                        'language': "en",
                        'journal_eissn': journal['eissn'],
                        'journal_pissn': journal['pissn'],
                        'category': journal['categories']
                    }).rename(doi)
            df = df.append(datadict) 
        df.to_pickle(filename)
        print(df.tail())
        return df
    
    if not re.search('JournalDisplayName', driver.current_url):
        print("Error to set Journal:", driver.current_url)
        return df, olddf
    
    # firs result
    df = processresult(df)        
    
    # iterate over the nexts
    while True:
        try:
            sleep(np.random.uniform(3,60))
            for nextbottom in driver.find_elements_by_css_selector("div.pagination a.al-nav-next"):
                nextbottom.click()
                break
            df = processresult(df)
        except:
            try:
                for nextbottom in driver.find_elements_by_css_selector("div.al-pageNumbers a.al-nav-next"):
                    nextbottom.click()
                    break
            except Exception as e:
                print("Error on next:", e)
                break
        
    return df, olddf
    

# Main

In [2]:
journals = pd.read_csv("doaj_journals.csv", sep="|")
journals = journals[journals['publisher'] == "Oxford University Press"]
journals = journals.to_dict('records')
print(journals)

NameError: name 'pd' is not defined

In [1]:


for jidx in range(len(journals)):
    filename = filename = 'data/journal_'+ journals[jidx]['title'].replace(" ", "_").replace(":", "").replace("&", "and") +'_'+version+'.pandas'
    
    # search for articles
    df, olddf = gedjournalarticles( journals[jidx], filename )
    
    # merg olddf and df
    df = pd.concat([df,olddf], axis=0)
    df.drop_duplicates(inplace=True)
    df.to_pickle(filename)

NameError: name 'journals' is not defined